# Padchest Unimodal

In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import ast
from utils import *
from sklearn import preprocessing, metrics
import json
from keras.backend import squeeze, expand_dims
from sklearn.metrics import roc_auc_score
from keras.models import load_model

Using TensorFlow backend.


In [2]:
#Pipeline Parameters
createEmbeddings = False
createFolds = True
onlyPhysicianLabels = True
projection = 'PA'
workers = 3
n_epochs = 80
image_loc = '/gpfs/ysm/home/kl533/krauthammer_partition/padchest_2019_small/images-224/'
fitVerbose = 0
with open('n_splits.json') as json_file:
    json_data = json.load(json_file)
n_splits = json_data['n_splits']

In [3]:
#word2vec parameters
w2v_params = {'vocabulary_size':10000, 'window_size':5,'filters':'','vector_dim':224}
#parameters
max_text_len = 200#this is the same size as the sequence length
vocabulary_size = 10000
embedding_output_dim = 224
shape=(224,224,1)
channels = shape[-1]

In [4]:
#build our clean csv
#%run -i 'python3 extract_cohort.py' '/gpfs/ysm/home/kl533/krauthammer_partition/padchest_2019_small/PADCHEST_chest_x_ray_images_labels_160K_01.02.19.csv' '/gpfs/ysm/home/kl533/krauthammer_partition/padchest_2019_small/PADCHEST_CLEAN.csv' '/gpfs/ysm/home/kl533/krauthammer_partition/padchest_2019_small/images-224/'
#load already setup csv, with clean labels
df = pd.read_csv('/gpfs/ysm/pi/krauthammer/kl533/padchest_2019_small/PADCHEST_CLEAN.csv')

#lets only pull the labels done by physicians:
if onlyPhysicianLabels:
    pattern = 'Physician'
    df = df[df['MethodLabel'] == pattern]

df['Report'] = df['Report'].apply(lambda x: x if type(x) == str else '')#fix any reports that are na

In [5]:
l = df.groupby('StudyID').first()['Clean_Labels'].apply(lambda x: ast.literal_eval(x)).to_list()
#list(set([item for sublist in l for item in sublist]))

In [6]:
label_counts = {}
for ll in l:
    for label in ll:
        if label in label_counts:
            label_counts[label] += 1
        else:
            label_counts[label] = 1
label_counts['other'] = 0
to_del = []
for k,v in label_counts.items():
    if v < 400:
        label_counts['other'] += v
        to_del.append(k)
for key in to_del: del label_counts[key]
#print(label_counts)
#print(len(label_counts))

In [7]:
label_counts

{'normal': 3056,
 'cardiomegaly': 753,
 'COPD signs': 1072,
 'infiltrates': 964,
 'aortic elongation': 621,
 'thoracic cage deformation': 1028,
 'atelectasis': 447,
 'pneumonia': 490,
 'air trapping': 509,
 'vertebral degenerative changes': 743,
 'other': 6364}

In [8]:
df['Final_Labels'] = df['Clean_Labels'].apply(lambda x: [t if t in label_counts.keys() else 'other' for t in ast.literal_eval(x)])

In [9]:
num_class = len(label_counts)

In [10]:
if createEmbeddings:
    print('Section 0.5: create embeddings')
    from w2v_keras import w2v_keras
    w2v = w2v_keras(**w2v_params)
    w2v.fit(df['Report'], epochs=20)
    w2v.save_embeddings('w2v_embeddings_spanish.json')

## Build our tokenizer to pass into generators

In [11]:
tokenizer = Tokenizer(num_words = 10000, lower=True, filters='')#make this a param
tokenizer.fit_on_texts(filter_sentences_spanish(df['Report'].tolist()))

filtering sentences
filtering sents and removing stopwords


In [12]:
mlb = preprocessing.MultiLabelBinarizer()
mlb.fit(df['Final_Labels'].to_list())

MultiLabelBinarizer(classes=None, sparse_output=False)

### Load Embeddings from JSON file and map tokens to embedding

In [13]:
#load embedding weights from file
location = 'w2v_embeddings_spanish.json'
import json
with open(location) as f:
    embs = json.load(f)
    emb = json.loads(embs)
embedding_weights = mapTokensToEmbedding(emb, tokenizer.word_index, 
                                         vocabulary_size)

In [14]:
from keras_applications.resnet_v2 import ResNet50V2
from keras_applications.densenet import DenseNet121
from keras.layers import GlobalAveragePooling2D, Dense
from keras import Sequential
def DenseNetModelFactory():
    model = DenseNet121(input_shape=shape, include_top=False, weights=None,backend=keras.backend,
        layers=keras.layers,
        models=keras.models,
        utils=keras.utils)
    return model

In [15]:
from keras.models import Model
from keras.layers import Dense, Reshape, concatenate, Lambda, Average, Maximum, Add, Multiply, Concatenate, BatchNormalization, Activation, GlobalAveragePooling2D, GlobalAveragePooling1D, Flatten
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.layers.embeddings import Embedding
from keras.engine.input_layer import Input
from keras.optimizers import Adam
from keras.activations import relu


def Generate_Model(model_type = 'LateFusion'):
    #set up our text input
    if model_type == 'LateFusion':
        input_text = Input(shape=(max_text_len,), name='text_input')
        embedding = Embedding(input_length=max_text_len, input_dim=vocabulary_size, output_dim=embedding_output_dim, 
                               name='embedding_layer', trainable=False, weights=[embedding_weights])
        text_embedding = embedding(input_text)
        #set up our image branch
        image_input = Input(shape, name='image_input')
        image_branch = DenseNetModelFactory()(image_input)
        image_branch = Flatten()(image_branch)
        image_branch = Dense(1024, activation='relu', name='dense_layer1_image_branch')(image_branch)
        image_branch = BatchNormalization()(image_branch)
        image_branch = Dropout(0.5)(image_branch)        
        image_branch = Dense(512, activation='relu', name='dense_layer2_image_branch')(image_branch)
        image_branch = BatchNormalization()(image_branch)
        image_branch = Dropout(0.5)(image_branch)
        image_branch_out = Dense(num_class, activation='sigmoid', name='image_branch_output')(image_branch)
        image_branch_model = Model(inputs=[image_input], outputs=[image_branch_out])
        #set up text branch
        text_branch = get_text_branch()(text_embedding)
        text_branch = Dense(1024, activation='relu', name='dense_layer1_text_branch')(text_branch)
        text_branch = BatchNormalization()(text_branch)
        text_branch = Dropout(0.5)(text_branch)        
        text_branch = Dense(512, activation='relu', name='dense_layer2_text_branch')(text_branch)
        text_branch = BatchNormalization()(text_branch)
        text_branch = Dropout(0.5)(text_branch)
        text_branch_out = Dense(num_class, activation='sigmoid', name='text_branch_output')(text_branch)
        text_branch_model = Model(inputs=[input_text], outputs=[text_branch_out])
        outputs = [model.outputs[0] for model in [text_branch_model, image_branch_model]]
        output = Average()(outputs)
        classificationModel = Model(inputs=[text_branch_model.input, image_branch_model.input], outputs=[output])
    if model_type == "EarlyFusion":
        input_text = Input(shape=(max_text_len,), name='text_input')
        embedding = Embedding(input_length=max_text_len, input_dim=vocabulary_size, output_dim=embedding_output_dim, 
                               name='embedding_layer', trainable=False, weights=[embedding_weights])
        embedded_text = embedding(input_text)
        #expand dims if channel = 1 as we need (height, width, channels) 
        #and a matrix with one channel only has 2 dims
        if channels == 1:
            expandedText = Lambda(lambda x: expand_dims(x, axis=-1))
            embedded_text = expandedText(embedded_text)
        else:
            embedded_text = Lambda(embedding_3Ch)(embedded_text)
        #set up our image input
        image_input = Input(shape, name='image_input')
        '''merge our data for early fusion. Essentially we are just concatenating our data together'''
        merged = concatenate([embedded_text, image_input], axis=1, name='merged')
        x = DenseNetModelFactory()(merged)
        x = Flatten()(x)
        x = Dense(1024, activation='relu', name='dense_layer1_earlyFusion')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)        
        x = Dense(512, activation='relu', name='dense_layer2_earlyFusion')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        output = Dense(num_class, activation='sigmoid', name='output')(x)
        classificationModel = Model(inputs=[input_text, image_input], outputs=[output])
    if model_type == "EarlyFusion_Wide":
        input_text = Input(shape=(max_text_len,), name='text_input')
        embedding = Embedding(input_length=max_text_len, input_dim=vocabulary_size, output_dim=embedding_output_dim, 
                               name='embedding_layer', trainable=False, weights=[embedding_weights])
        embedded_text = embedding(input_text)
        #set up our image input
        image_input = Input(shape, name='image_input')
        image_input_reshaped = Reshape((shape[0]*channels,shape[0]))(image_input)#224 is because we have a 224x224 image with 2 channels, 224x1 channels = 224
        '''merge our data for early fusion. Essentially we are just concatenating our data together'''
        merged = concatenate([embedded_text, image_input_reshaped], axis=1, name='merged')
        x = get_text_branch()(merged)
        x = Dense(1024, activation='relu', name='dense_layer1')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)        
        x = Dense(512, activation='relu', name='dense_layer2')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        output = Dense(num_class, activation='sigmoid', name='output')(x)
        classificationModel = Model(inputs=[input_text, image_input], outputs=[output])        
    if model_type == 'ModelFusion':
        input_text = Input(shape=(max_text_len,), name='text_input')
        embedding = Embedding(input_length=max_text_len, input_dim=vocabulary_size, output_dim=embedding_output_dim, 
                               name='embedding_layer', trainable=False, weights=[embedding_weights])
        text_embedding = embedding(input_text)
        image_input = Input(shape, name='image_input')
        txt = get_text_branch()            
        #set up our image branch
        image_branch = DenseNetModelFactory()(image_input)
        image_branch = Flatten()(image_branch)
        #set up text branch
        text_branch = (txt)(text_embedding)
        #add some dense layers as test
        x = concatenate([image_branch, text_branch])
        x = Dense(1024, activation='relu', name='dense_layer1_modelFusion')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)        
        x = Dense(512, activation='relu', name='dense_layer2_modelFusion')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        #add some dense layers as test
        output = Dense(num_class, activation='sigmoid', name='output')(x)
        classificationModel = Model(inputs=[input_text, image_input], outputs=[output])
    if model_type == "text_1d":#text_1d only
        input_text = Input(shape=(max_text_len,), name='text_input')
        embedding = Embedding(input_length=max_text_len, input_dim=vocabulary_size, output_dim=embedding_output_dim, 
                               name='embedding_layer', trainable=False, weights=[embedding_weights])
        embedded_text = embedding(input_text)
        x = get_text_branch()(embedded_text)
        x = Dense(1024, activation='relu', name='dense_layer1_text_branch')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)        
        x = Dense(512, activation='relu', name='dense_layer2_text_branch')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        output = Dense(num_class, activation='sigmoid', name='text_output')(x)
        classificationModel = Model(inputs=[input_text], outputs=[output])
    if model_type == "image":#image only
        image_input = Input(shape, name='image_input')
        x = DenseNetModelFactory()(image_input)
        x = Flatten()(x)
        x = Dense(1024, activation='relu', name='dense_layer1_image_branch')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)        
        x = Dense(512, activation='relu', name='dense_layer2_image_branch')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        output = Dense(num_class, activation='sigmoid', name='image_output')(x)
        classificationModel = Model(inputs=[image_input], outputs=[output])
    return classificationModel

# Set up environment for Models

In [16]:
#split into train, valid, test of 80-10-10
if createFolds:
    #lets split the arrays
    splitDfs = np.array_split(df, n_splits)
    splits = {}
    for i, split in enumerate(splitDfs):
        train, valid, test = np.split(split[split['Projection'] == projection].sample(frac=1.), [int(.8*len(split[split['Projection'] == projection])), int(.9*len(split[split['Projection'] == projection]))])    
        splits[i] = {}
        splits[i]['train'] = train.index.tolist()
        splits[i]['valid'] = valid.index.tolist()
        splits[i]['test'] = test.index.tolist()
    with open('padchestFolds_{}.json'.format(projection), 'w') as fp:
        json.dump(splits, fp)

In [17]:
#load folds
with open('padchestFolds_{}.json'.format(projection), 'r') as fp:
    data = json.load(fp)

In [18]:
chunkList = []
for ix,chunk in data.items():
    chunkList.append(chunk['train'])
    chunkList.append(chunk['valid'])
    chunkList.append(chunk['test'])
for i in range(len(chunkList)):
    for j in range(i+1, len(chunkList)):
        assert len(set(chunkList[i]).intersection(chunkList[j])) == 0, "You have an element from one chunk in another! Each chunk should have unique indicies"

In [19]:
from keras import backend as K
from sklearn.metrics import hamming_loss
from sklearn.metrics import roc_auc_score
import tensorflow as tf
def full_multi_label_metric(y_true, y_pred):
    comp = K.equal(y_true, K.round(y_pred))
    return K.cast(K.all(comp, axis=-1), K.floatx())
def Hamming_loss(y_true, y_pred):
    return K.cast(hamming_loss(K.eval(y_true), K.round(K.eval(y_pred))), K.floatx())
def auc( y_true, y_pred ) :
    score = tf.py_func( lambda y_true, y_pred : roc_auc_score( y_true, y_pred, average='weighted', sample_weight=None).astype('float32'),
                        [y_true, y_pred],
                        'float32',
                        stateful=False,
                        name='sklearnAUC' )
    return score

In [20]:
#call back AUC score
#callback code: https://github.com/keras-team/keras/issues/10472#issuecomment-472543538
from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback
class AUC(Callback):
    
    def __init__(self, val_data, batch_size = 8):
        super().__init__()
        self.validation_data = val_data
        self.batch_size = batch_size
    
    def on_train_begin(self, logs={}):
        #print(self.validation_data)
        self.val_f1s = []
        self.val_recalls = []
        self.val_precisions = []
        self._val_auc = []
        
    def on_epoch_end(self, epoch, logs={}):
        batches = len(self.validation_data)
        total = batches * self.batch_size
        y_shape = len(self.validation_data.mlb.classes_)
        val_pred = np.zeros((total, y_shape))
        val_true = np.zeros((total, y_shape))
        
        for batch in range(batches):
            if len(self.validation_data.modality) == 1:
                xVal, yVal = self.validation_data.__getitem__(batch)
                val_pred[batch * self.batch_size : (batch+1) * self.batch_size] = np.asarray(self.model.predict(xVal))
            elif len(self.validation_data.modality) == 2:
                [xValTxt, xValImg], yVal = self.validation_data.__getitem__(batch)
                val_pred[batch * self.batch_size : (batch+1) * self.batch_size] = np.asarray(self.model.predict([xValTxt, xValImg]))
            val_true[batch * self.batch_size : (batch+1) * self.batch_size] = yVal
            
        #val_pred = np.squeeze(val_pred)
        _val_auc = roc_auc_score(val_true, val_pred, average='weighted')#something has to be fixed
        print('Validation AUC: {}'.format(_val_auc))
        self._val_auc.append(_val_auc)
        
        return

# Setup our Experiments

In [21]:
from keras.optimizers import Adam
import tensorflow as tf
def optimizerFactory():
    return Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False)

In [22]:
import math
# learning rate schedule
def step_decay(epoch):
    initial_lrate = 0.0001
    drop = 0.5
    epochs_drop = 10.0
    lrate = initial_lrate * math.pow(drop,  
                math.floor((1+epoch)/epochs_drop))
    print('Lowering LR to {}'.format(lrate))
    return lrate

In [23]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
def getCheckpointCallback(name):
    return ModelCheckpoint(name, monitor='full_multi_label_metric', verbose=1, save_best_only=True, save_weights_only=False, mode='max', period=1)

In [24]:
#reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6, verbose=1)
#early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=15, verbose=0, mode='auto', baseline=None)

# Train images

In [25]:
#scheduled learning rate
lrate = LearningRateScheduler(step_decay)

In [26]:
params = {'dim': (224,224),
              'batch_size': 8,
              'n_channels': 1,
              'loc': image_loc,
              'shuffle': False,
              'modality': ['image'],
              'tokenizer':tokenizer,
               'mlb': mlb}
weightedAUCList = []
AUCList = []
for ix, split in data.items():
    K.clear_session()
    print('====================Starting Split Number {}===================='.format(ix))
    model = Generate_Model(model_type='image')#create model
    model.compile(optimizer=optimizerFactory(), loss='binary_crossentropy', metrics=[full_multi_label_metric])
    
    train_generator = padchestDataGenerator_DF(df.iloc[split['train']].reset_index(drop=True), **params)
    valid_generator = padchestDataGenerator_DF(df.iloc[split['valid']].reset_index(drop=True), **params)
    test_generator = padchestDataGenerator_DF(df.iloc[split['test']].reset_index(drop=True), **params)
    
    #checkpoint
    filename = 'imageOnly_{}_split_{}.hdf5'.format(projection,ix)
    checkpoint = getCheckpointCallback(filename)
    model.fit_generator(train_generator, steps_per_epoch=len(train_generator), validation_data=valid_generator, 
                        validation_steps=len(valid_generator), workers=workers, epochs=n_epochs, verbose=fitVerbose, class_weight=train_generator.labels_weights, 
                        callbacks=[checkpoint,lrate, AUC(valid_generator)])
    y_hat = model.predict_generator(test_generator, steps=len(test_generator), workers=workers)
    auc = roc_auc_score(test_generator.mlb.transform(df.iloc[split['test']].iloc[:len(y_hat)]['Final_Labels']), y_hat, average=None)
    AUCList.append(auc)
    weighted_auc = roc_auc_score(test_generator.mlb.transform(df.iloc[split['test']].iloc[:len(y_hat)]['Final_Labels']), y_hat, average='weighted')
    weightedAUCList.append(weighted_auc)
    print('Weighted AUC score for split {}: {}'.format(ix,weighted_auc))
AUC_Scores = pd.DataFrame({'Label': test_generator.mlb.classes_, 'Score': np.mean(AUCList, axis=0)}, columns=['Label', 'Score'])
AUC_Scores = AUC_Scores.append({'Label': 'weighted_auc', 'Score':np.mean(weightedAUCList, axis=0)}, ignore_index=True)#to csv    
AUC_Scores.to_csv('AUC_scores_imageOnly_{}'.format(projection))

W0301 03:54:51.764790 47597341553536 deprecation_wrapper.py:119] From /gpfs/ysm/project/kl533/conda_envs/dlnn/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:95: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.

W0301 03:54:51.766256 47597341553536 deprecation_wrapper.py:119] From /gpfs/ysm/project/kl533/conda_envs/dlnn/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:98: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0301 03:54:51.793039 47597341553536 deprecation_wrapper.py:119] From /gpfs/ysm/project/kl533/conda_envs/dlnn/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:102: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0301 03:54:51.794197 47597341553536 deprecation_wrapper.py:119] From /gpfs/ysm/project/kl533/conda_envs/dlnn/lib/python3.6/site-packages/keras/backend/tens

====================Starting Split Number 0====================


W0301 03:54:53.167625 47597341553536 deprecation_wrapper.py:119] From /gpfs/ysm/project/kl533/conda_envs/dlnn/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:1834: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.

W0301 03:54:53.225205 47597341553536 deprecation_wrapper.py:119] From /gpfs/ysm/project/kl533/conda_envs/dlnn/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0301 03:54:54.223280 47597341553536 deprecation_wrapper.py:119] From /gpfs/ysm/project/kl533/conda_envs/dlnn/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3980: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.

W0301 03:55:09.474051 47597341553536 deprecation.py:506] From /gpfs/ysm/project/kl533/conda_envs/dlnn/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.

Encoding Labels
Encoding Labels
Encoding Labels
Lowering LR to 0.0001

Epoch 00001: full_multi_label_metric improved from -inf to 0.00397, saving model to imageOnly_PA_split_0.hdf5
Validation AUC: 0.6122184192018193
Lowering LR to 0.0001

Epoch 00002: full_multi_label_metric improved from 0.00397 to 0.03419, saving model to imageOnly_PA_split_0.hdf5
Validation AUC: 0.6793105141616443
Lowering LR to 0.0001

Epoch 00003: full_multi_label_metric improved from 0.03419 to 0.10791, saving model to imageOnly_PA_split_0.hdf5
Validation AUC: 0.6909555535977735
Lowering LR to 0.0001

Epoch 00004: full_multi_label_metric improved from 0.10791 to 0.16361, saving model to imageOnly_PA_split_0.hdf5
Validation AUC: 0.7247060295294036
Lowering LR to 0.0001

Epoch 00005: full_multi_label_metric improved from 0.16361 to 0.18391, saving model to imageOnly_PA_split_0.hdf5
Validation AUC: 0.7231978892292426
Lowering LR to 0.0001

Epoch 00006: full_multi_label_metric improved from 0.18391 to 0.19658, saving

In [27]:
'''for key in sorted(train_generator.labels_count):
    print('{}:{}'.format(key,train_generator.labels_count[key]))
print(len(train_generator.labels_count))
print('====================================')
for key in sorted(valid_generator.labels_count):
    print('{}:{}'.format(key,valid_generator.labels_count[key]))
print(len(valid_generator.labels_count))
print('====================================')
for key in sorted(test_generator.labels_count):
    print('{}:{}'.format(key,test_generator.labels_count[key]))
print(len(test_generator.labels_count))'''

"for key in sorted(train_generator.labels_count):\n    print('{}:{}'.format(key,train_generator.labels_count[key]))\nprint(len(train_generator.labels_count))\nprint('====================================')\nfor key in sorted(valid_generator.labels_count):\n    print('{}:{}'.format(key,valid_generator.labels_count[key]))\nprint(len(valid_generator.labels_count))\nprint('====================================')\nfor key in sorted(test_generator.labels_count):\n    print('{}:{}'.format(key,test_generator.labels_count[key]))\nprint(len(test_generator.labels_count))"

## Text only

In [28]:
params = {'dim': (224,224),
              'batch_size': 8,
              'n_channels': 1,
              'loc': image_loc,
              'shuffle': False,
              'modality': ['text'],
              'tokenizer':tokenizer,
               'mlb': mlb}
weightedAUCList = []
AUCList = []
for ix, split in data.items():
    K.clear_session()
    print('====================Starting Split Number {}===================='.format(ix))
    model = Generate_Model(model_type='text_1d')#create model
    model.compile(optimizer=optimizerFactory(), loss='binary_crossentropy', metrics=[full_multi_label_metric])
    
    train_generator = padchestDataGenerator_DF(df.iloc[split['train']].reset_index(drop=True), **params)
    valid_generator = padchestDataGenerator_DF(df.iloc[split['valid']].reset_index(drop=True), **params)
    test_generator = padchestDataGenerator_DF(df.iloc[split['test']].reset_index(drop=True), **params)
    #checkpoint
    filename = 'textOnly_{}_split_{}.hdf5'.format(projection,ix)
    checkpoint = getCheckpointCallback(filename)
    model.fit_generator(train_generator, steps_per_epoch=len(train_generator), validation_data=valid_generator, 
                        validation_steps=len(valid_generator), workers=workers, epochs=n_epochs, verbose=fitVerbose, class_weight=train_generator.labels_weights, 
                        callbacks=[checkpoint,lrate, AUC(valid_generator)])
    y_hat = model.predict_generator(test_generator, steps=len(test_generator), workers=workers)
    auc = roc_auc_score(test_generator.mlb.transform(df.iloc[split['test']].iloc[:len(y_hat)]['Final_Labels']), y_hat, average=None)
    AUCList.append(auc)
    weighted_auc = roc_auc_score(test_generator.mlb.transform(df.iloc[split['test']].iloc[:len(y_hat)]['Final_Labels']), y_hat, average='weighted')
    weightedAUCList.append(weighted_auc)
    print('Weighted AUC score for split {}: {}'.format(ix,weighted_auc))
AUC_Scores = pd.DataFrame({'Label': test_generator.mlb.classes_, 'Score': np.mean(AUCList, axis=0)}, columns=['Label', 'Score'])
AUC_Scores = AUC_Scores.append({'Label': 'weighted_auc', 'Score':np.mean(weightedAUCList, axis=0)}, ignore_index=True)#to csv    
AUC_Scores.to_csv('AUC_scores_textOnly_{}'.format(projection))

====================Starting Split Number 0====================
Encoding Labels
You have passed in a tokenizer, we will just convert to sequences
filtering sentences
filtering sents and removing stopwords
Encoding Labels
You have passed in a tokenizer, we will just convert to sequences
filtering sentences
filtering sents and removing stopwords
Encoding Labels
You have passed in a tokenizer, we will just convert to sequences
filtering sentences
filtering sents and removing stopwords
Lowering LR to 0.0001

Epoch 00001: full_multi_label_metric improved from -inf to 0.00977, saving model to textOnly_PA_split_0.hdf5
Validation AUC: 0.7759700067964073
Lowering LR to 0.0001

Epoch 00002: full_multi_label_metric improved from 0.00977 to 0.08471, saving model to textOnly_PA_split_0.hdf5
Validation AUC: 0.7988968798830886
Lowering LR to 0.0001

Epoch 00003: full_multi_label_metric improved from 0.08471 to 0.24100, saving model to textOnly_PA_split_0.hdf5
Validation AUC: 0.8318469922640525
Loweri

# Train multimodal Models

In [29]:
params = {'dim': (224,224),
              'batch_size': 8,
              'n_channels': 1,
              'loc': '/gpfs/ysm/home/kl533/krauthammer_partition/padchest_2019_small/images-224/',
              'shuffle': False,
              'modality': ['image', 'text'],
              'tokenizer':tokenizer,
               'mlb': mlb}

## Model Fusion Randomly Initialized

In [30]:
weightedAUCList = []
AUCList = []
for ix, split in data.items():
    K.clear_session()
    print('====================Starting Split Number {}===================='.format(ix))
    model = Generate_Model(model_type='ModelFusion')#create model
    model.compile(optimizer=optimizerFactory(), loss='binary_crossentropy', metrics=[full_multi_label_metric])
    
    train_generator = padchestDataGenerator_DF(df.iloc[split['train']].reset_index(drop=True), **params)
    valid_generator = padchestDataGenerator_DF(df.iloc[split['valid']].reset_index(drop=True), **params)
    test_generator = padchestDataGenerator_DF(df.iloc[split['test']].reset_index(drop=True), **params)
    #checkpoint
    filename = 'modelFusion_{}_split_{}.hdf5'.format(projection,ix)#'imageOnly_lat.hdf5'
    checkpoint = getCheckpointCallback(filename)
    model.fit_generator(train_generator, steps_per_epoch=len(train_generator), validation_data=valid_generator, 
                        validation_steps=len(valid_generator), workers=workers, epochs=n_epochs, verbose=fitVerbose, class_weight=train_generator.labels_weights, 
                        callbacks=[checkpoint,lrate, AUC(valid_generator)])
    y_hat = model.predict_generator(test_generator, steps=len(test_generator), workers=workers)
    auc = roc_auc_score(test_generator.mlb.transform(df.iloc[split['test']].iloc[:len(y_hat)]['Final_Labels']), y_hat, average=None)
    AUCList.append(auc)
    weighted_auc = roc_auc_score(test_generator.mlb.transform(df.iloc[split['test']].iloc[:len(y_hat)]['Final_Labels']), y_hat, average='weighted')
    weightedAUCList.append(weighted_auc)
    print('Weighted AUC score for split {}: {}'.format(ix,weighted_auc))
AUC_Scores = pd.DataFrame({'Label': test_generator.mlb.classes_, 'Score': np.mean(AUCList, axis=0)}, columns=['Label', 'Score'])
AUC_Scores = AUC_Scores.append({'Label': 'weighted_auc', 'Score':np.mean(weightedAUCList, axis=0)}, ignore_index=True)#to csv    
AUC_Scores.to_csv('AUC_scores_ModelFusion_{}'.format(projection))

====================Starting Split Number 0====================
Encoding Labels
You have passed in a tokenizer, we will just convert to sequences
filtering sentences
filtering sents and removing stopwords
Encoding Labels
You have passed in a tokenizer, we will just convert to sequences
filtering sentences
filtering sents and removing stopwords
Encoding Labels
You have passed in a tokenizer, we will just convert to sequences
filtering sentences
filtering sents and removing stopwords
Lowering LR to 0.0001

Epoch 00001: full_multi_label_metric improved from -inf to 0.00733, saving model to modelFusion_PA_split_0.hdf5
Validation AUC: 0.7752638695777186
Lowering LR to 0.0001

Epoch 00002: full_multi_label_metric improved from 0.00733 to 0.07845, saving model to modelFusion_PA_split_0.hdf5
Validation AUC: 0.8124004521313167
Lowering LR to 0.0001

Epoch 00003: full_multi_label_metric improved from 0.07845 to 0.23230, saving model to modelFusion_PA_split_0.hdf5
Validation AUC: 0.86580393407233

## Model Fusion Pretrained

In [31]:
weightedAUCList = []
AUCList = []
for ix, split in data.items():
    K.clear_session()
    print('====================Starting Split Number {}===================='.format(ix))
    model = Generate_Model(model_type='ModelFusion')#create model
    model.compile(optimizer=optimizerFactory(), loss='binary_crossentropy', metrics=[full_multi_label_metric])
    
    train_generator = padchestDataGenerator_DF(df.iloc[split['train']].reset_index(drop=True), **params)
    valid_generator = padchestDataGenerator_DF(df.iloc[split['valid']].reset_index(drop=True), **params)
    test_generator = padchestDataGenerator_DF(df.iloc[split['test']].reset_index(drop=True), **params)
    #checkpoint
    filename = 'modelFusion_PT_{}_split_{}.hdf5'.format(projection,ix)
    checkpoint = getCheckpointCallback(filename)
    
    #Load weights
    model.layers[2].set_weights(load_model('imageOnly_{}_split_{}.hdf5'.format(projection,ix),compile=False).layers[1].get_weights())#image
    model.layers[5].set_weights(load_model('textOnly_{}_split_{}.hdf5'.format(projection,ix),compile=False).layers[2].get_weights())#text
    
    #Fit Model
    model.fit_generator(train_generator, steps_per_epoch=len(train_generator), validation_data=valid_generator, 
                        validation_steps=len(valid_generator), workers=workers, epochs=n_epochs, verbose=fitVerbose, class_weight=train_generator.labels_weights, 
                        callbacks=[checkpoint,lrate, AUC(valid_generator)])
    y_hat = model.predict_generator(test_generator, steps=len(test_generator), workers=workers)
    auc = roc_auc_score(test_generator.mlb.transform(df.iloc[split['test']].iloc[:len(y_hat)]['Final_Labels']), y_hat, average=None)
    AUCList.append(auc)
    weighted_auc = roc_auc_score(test_generator.mlb.transform(df.iloc[split['test']].iloc[:len(y_hat)]['Final_Labels']), y_hat, average='weighted')
    weightedAUCList.append(weighted_auc)
    print('Weighted AUC score for split {}: {}'.format(ix,weighted_auc))
AUC_Scores = pd.DataFrame({'Label': test_generator.mlb.classes_, 'Score': np.mean(AUCList, axis=0)}, columns=['Label', 'Score'])
AUC_Scores = AUC_Scores.append({'Label': 'weighted_auc', 'Score':np.mean(weightedAUCList, axis=0)}, ignore_index=True)#to csv    
AUC_Scores.to_csv('AUC_scores_ModelFusion_PT_{}'.format(projection))

====================Starting Split Number 0====================
Encoding Labels
You have passed in a tokenizer, we will just convert to sequences
filtering sentences
filtering sents and removing stopwords
Encoding Labels
You have passed in a tokenizer, we will just convert to sequences
filtering sentences
filtering sents and removing stopwords
Encoding Labels
You have passed in a tokenizer, we will just convert to sequences
filtering sentences
filtering sents and removing stopwords
Lowering LR to 0.0001

Epoch 00001: full_multi_label_metric improved from -inf to 0.01389, saving model to modelFusion_PT_PA_split_0.hdf5
Validation AUC: 0.890046140056836
Lowering LR to 0.0001

Epoch 00002: full_multi_label_metric improved from 0.01389 to 0.10821, saving model to modelFusion_PT_PA_split_0.hdf5
Validation AUC: 0.9269861768123374
Lowering LR to 0.0001

Epoch 00003: full_multi_label_metric improved from 0.10821 to 0.29594, saving model to modelFusion_PT_PA_split_0.hdf5
Validation AUC: 0.939592

## Late Fusion Randomly Initialized

In [32]:
weightedAUCList = []
AUCList = []
for ix, split in data.items():
    K.clear_session()
    print('====================Starting Split Number {}===================='.format(ix))
    model = Generate_Model(model_type='LateFusion')#create model
    model.compile(optimizer=optimizerFactory(), loss='binary_crossentropy', metrics=[full_multi_label_metric])
    
    train_generator = padchestDataGenerator_DF(df.iloc[split['train']].reset_index(drop=True), **params)
    valid_generator = padchestDataGenerator_DF(df.iloc[split['valid']].reset_index(drop=True), **params)
    test_generator = padchestDataGenerator_DF(df.iloc[split['test']].reset_index(drop=True), **params)
    #checkpoint
    filename = 'lateFusion_{}_split_{}.hdf5'.format(projection,ix)#'imageOnly_lat.hdf5'
    checkpoint = getCheckpointCallback(filename)
    model.fit_generator(train_generator, steps_per_epoch=len(train_generator), validation_data=valid_generator, 
                        validation_steps=len(valid_generator), workers=workers, epochs=n_epochs, verbose=fitVerbose, class_weight=train_generator.labels_weights, 
                        callbacks=[checkpoint,lrate, AUC(valid_generator)])
    y_hat = model.predict_generator(test_generator, steps=len(test_generator), workers=workers)
    auc = roc_auc_score(test_generator.mlb.transform(df.iloc[split['test']].iloc[:len(y_hat)]['Final_Labels']), y_hat, average=None)
    AUCList.append(auc)
    weighted_auc = roc_auc_score(test_generator.mlb.transform(df.iloc[split['test']].iloc[:len(y_hat)]['Final_Labels']), y_hat, average='weighted')
    weightedAUCList.append(weighted_auc)
    print('Weighted AUC score for split {}: {}'.format(ix,weighted_auc))
AUC_Scores = pd.DataFrame({'Label': test_generator.mlb.classes_, 'Score': np.mean(AUCList, axis=0)}, columns=['Label', 'Score'])
AUC_Scores = AUC_Scores.append({'Label': 'weighted_auc', 'Score':np.mean(weightedAUCList, axis=0)}, ignore_index=True)#to csv    
AUC_Scores.to_csv('AUC_scores_lateFusion_{}'.format(projection))

====================Starting Split Number 0====================
Encoding Labels
You have passed in a tokenizer, we will just convert to sequences
filtering sentences
filtering sents and removing stopwords
Encoding Labels
You have passed in a tokenizer, we will just convert to sequences
filtering sentences
filtering sents and removing stopwords
Encoding Labels
You have passed in a tokenizer, we will just convert to sequences
filtering sentences
filtering sents and removing stopwords
Lowering LR to 0.0001

Epoch 00001: full_multi_label_metric improved from -inf to 0.04197, saving model to lateFusion_PA_split_0.hdf5
Validation AUC: 0.7785030824222612
Lowering LR to 0.0001

Epoch 00002: full_multi_label_metric improved from 0.04197 to 0.18147, saving model to lateFusion_PA_split_0.hdf5
Validation AUC: 0.8096275297050022
Lowering LR to 0.0001

Epoch 00003: full_multi_label_metric improved from 0.18147 to 0.29197, saving model to lateFusion_PA_split_0.hdf5
Validation AUC: 0.8512563564724381


## Late Fusion Pretrained

In [33]:
weightedAUCList = []
AUCList = []
for ix, split in data.items():
    K.clear_session()
    print('====================Starting Split Number {}===================='.format(ix))
    model = Generate_Model(model_type='LateFusion')#create model
    model.compile(optimizer=optimizerFactory(), loss='binary_crossentropy', metrics=[full_multi_label_metric])
    
    train_generator = padchestDataGenerator_DF(df.iloc[split['train']].reset_index(drop=True), **params)
    valid_generator = padchestDataGenerator_DF(df.iloc[split['valid']].reset_index(drop=True), **params)
    test_generator = padchestDataGenerator_DF(df.iloc[split['test']].reset_index(drop=True), **params)
    #checkpoint
    filename = 'lateFusion_PT_{}_split_{}.hdf5'.format(projection,ix)
    checkpoint = getCheckpointCallback(filename)
    
    #Load weights: This will only load the feature extractors, not the 2 dense layers at the end.
    model.layers[3].set_weights(load_model('imageOnly_{}_split_{}.hdf5'.format(projection,ix),compile=False).layers[1].get_weights())#image
    model.layers[4].set_weights(load_model('textOnly_{}_split_{}.hdf5'.format(projection,ix),compile=False).layers[2].get_weights())#text
    
    #Fit Model
    model.fit_generator(train_generator, steps_per_epoch=len(train_generator), validation_data=valid_generator, 
                        validation_steps=len(valid_generator), workers=workers, epochs=n_epochs, verbose=fitVerbose, class_weight=train_generator.labels_weights, 
                        callbacks=[checkpoint,lrate, AUC(valid_generator)])
    y_hat = model.predict_generator(test_generator, steps=len(test_generator), workers=workers)
    auc = roc_auc_score(test_generator.mlb.transform(df.iloc[split['test']].iloc[:len(y_hat)]['Final_Labels']), y_hat, average=None)
    AUCList.append(auc)
    weighted_auc = roc_auc_score(test_generator.mlb.transform(df.iloc[split['test']].iloc[:len(y_hat)]['Final_Labels']), y_hat, average='weighted')
    weightedAUCList.append(weighted_auc)
    print('Weighted AUC score for split {}: {}'.format(ix,weighted_auc))
AUC_Scores = pd.DataFrame({'Label': test_generator.mlb.classes_, 'Score': np.mean(AUCList, axis=0)}, columns=['Label', 'Score'])
AUC_Scores = AUC_Scores.append({'Label': 'weighted_auc', 'Score':np.mean(weightedAUCList, axis=0)}, ignore_index=True)#to csv    
AUC_Scores.to_csv('AUC_scores_lateFusion_PT_{}'.format(projection))

====================Starting Split Number 0====================
Encoding Labels
You have passed in a tokenizer, we will just convert to sequences
filtering sentences
filtering sents and removing stopwords
Encoding Labels
You have passed in a tokenizer, we will just convert to sequences
filtering sentences
filtering sents and removing stopwords
Encoding Labels
You have passed in a tokenizer, we will just convert to sequences
filtering sentences
filtering sents and removing stopwords
Lowering LR to 0.0001

Epoch 00001: full_multi_label_metric improved from -inf to 0.04686, saving model to lateFusion_PT_PA_split_0.hdf5
Validation AUC: 0.9166832140042581
Lowering LR to 0.0001

Epoch 00002: full_multi_label_metric improved from 0.04686 to 0.23764, saving model to lateFusion_PT_PA_split_0.hdf5
Validation AUC: 0.9238844831840631
Lowering LR to 0.0001

Epoch 00003: full_multi_label_metric improved from 0.23764 to 0.40293, saving model to lateFusion_PT_PA_split_0.hdf5
Validation AUC: 0.93468714

## Early Fusion

In [34]:
weightedAUCList = []
AUCList = []
for ix, split in data.items():
    K.clear_session()
    print('====================Starting Split Number {}===================='.format(ix))
    model = Generate_Model(model_type='EarlyFusion')
    model.compile(optimizer=optimizerFactory(), loss='binary_crossentropy', metrics=[full_multi_label_metric])
    
    train_generator = padchestDataGenerator_DF(df.iloc[split['train']].reset_index(drop=True), **params)
    valid_generator = padchestDataGenerator_DF(df.iloc[split['valid']].reset_index(drop=True), **params)
    test_generator = padchestDataGenerator_DF(df.iloc[split['test']].reset_index(drop=True), **params)
    #checkpoint
    filename = 'earlyFusion_{}_split_{}.hdf5'.format(projection,ix)#'imageOnly_lat.hdf5'
    checkpoint = getCheckpointCallback(filename)
    model.fit_generator(train_generator, steps_per_epoch=len(train_generator), validation_data=valid_generator, 
                        validation_steps=len(valid_generator), workers=workers, epochs=n_epochs, verbose=fitVerbose, class_weight=train_generator.labels_weights, 
                        callbacks=[checkpoint,lrate, AUC(valid_generator)])
    y_hat = model.predict_generator(test_generator, steps=len(test_generator), workers=workers)
    auc = roc_auc_score(test_generator.mlb.transform(df.iloc[split['test']].iloc[:len(y_hat)]['Final_Labels']), y_hat, average=None)
    AUCList.append(auc)
    weighted_auc = roc_auc_score(test_generator.mlb.transform(df.iloc[split['test']].iloc[:len(y_hat)]['Final_Labels']), y_hat, average='weighted')
    weightedAUCList.append(weighted_auc)
    print('Weighted AUC score for split {}: {}'.format(ix,weighted_auc))
AUC_Scores = pd.DataFrame({'Label': test_generator.mlb.classes_, 'Score': np.mean(AUCList, axis=0)}, columns=['Label', 'Score'])
AUC_Scores = AUC_Scores.append({'Label': 'weighted_auc', 'Score':np.mean(weightedAUCList, axis=0)}, ignore_index=True)#to csv    
AUC_Scores.to_csv('AUC_scores_earlyFusion_{}'.format(projection))

====================Starting Split Number 0====================
Encoding Labels
You have passed in a tokenizer, we will just convert to sequences
filtering sentences
filtering sents and removing stopwords
Encoding Labels
You have passed in a tokenizer, we will just convert to sequences
filtering sentences
filtering sents and removing stopwords
Encoding Labels
You have passed in a tokenizer, we will just convert to sequences
filtering sentences
filtering sents and removing stopwords
Lowering LR to 0.0001

Epoch 00001: full_multi_label_metric improved from -inf to 0.01160, saving model to earlyFusion_PA_split_0.hdf5
Validation AUC: 0.7700144411153766
Lowering LR to 0.0001

Epoch 00002: full_multi_label_metric improved from 0.01160 to 0.08501, saving model to earlyFusion_PA_split_0.hdf5
Validation AUC: 0.8554921446278595
Lowering LR to 0.0001

Epoch 00003: full_multi_label_metric improved from 0.08501 to 0.23855, saving model to earlyFusion_PA_split_0.hdf5
Validation AUC: 0.90542123171302

## Early Fusion (Wide)

In [35]:
weightedAUCList = []
AUCList = []
for ix, split in data.items():
    K.clear_session()
    print('====================Starting Split Number {}===================='.format(ix))
    model = Generate_Model(model_type='EarlyFusion_Wide')
    model.compile(optimizer=optimizerFactory(), loss='binary_crossentropy', metrics=[full_multi_label_metric])
    
    train_generator = padchestDataGenerator_DF(df.iloc[split['train']].reset_index(drop=True), **params)
    valid_generator = padchestDataGenerator_DF(df.iloc[split['valid']].reset_index(drop=True), **params)
    test_generator = padchestDataGenerator_DF(df.iloc[split['test']].reset_index(drop=True), **params)
    #checkpoint
    filename = 'earlyFusion_wide_{}_split_{}.hdf5'.format(projection,ix)#'imageOnly_lat.hdf5'
    checkpoint = getCheckpointCallback(filename)
    model.fit_generator(train_generator, steps_per_epoch=len(train_generator), validation_data=valid_generator, 
                        validation_steps=len(valid_generator), workers=workers, epochs=n_epochs, verbose=fitVerbose, class_weight=train_generator.labels_weights, 
                        callbacks=[checkpoint,lrate, AUC(valid_generator)])
    y_hat = model.predict_generator(test_generator, steps=len(test_generator), workers=workers)
    auc = roc_auc_score(test_generator.mlb.transform(df.iloc[split['test']].iloc[:len(y_hat)]['Final_Labels']), y_hat, average=None)
    AUCList.append(auc)
    weighted_auc = roc_auc_score(test_generator.mlb.transform(df.iloc[split['test']].iloc[:len(y_hat)]['Final_Labels']), y_hat, average='weighted')
    weightedAUCList.append(weighted_auc)
    print('Weighted AUC score for split {}: {}'.format(ix,weighted_auc))
AUC_Scores = pd.DataFrame({'Label': test_generator.mlb.classes_, 'Score': np.mean(AUCList, axis=0)}, columns=['Label', 'Score'])
AUC_Scores = AUC_Scores.append({'Label': 'weighted_auc', 'Score':np.mean(weightedAUCList, axis=0)}, ignore_index=True)#to csv    
AUC_Scores.to_csv('AUC_scores_earlyFusion_wide_{}'.format(projection))

====================Starting Split Number 0====================
Encoding Labels
You have passed in a tokenizer, we will just convert to sequences
filtering sentences
filtering sents and removing stopwords
Encoding Labels
You have passed in a tokenizer, we will just convert to sequences
filtering sentences
filtering sents and removing stopwords
Encoding Labels
You have passed in a tokenizer, we will just convert to sequences
filtering sentences
filtering sents and removing stopwords
Lowering LR to 0.0001

Epoch 00001: full_multi_label_metric improved from -inf to 0.00687, saving model to earlyFusion_wide_PA_split_0.hdf5
Validation AUC: 0.7514203389089252
Lowering LR to 0.0001

Epoch 00002: full_multi_label_metric improved from 0.00687 to 0.05708, saving model to earlyFusion_wide_PA_split_0.hdf5
Validation AUC: 0.8008940211557797
Lowering LR to 0.0001

Epoch 00003: full_multi_label_metric improved from 0.05708 to 0.20009, saving model to earlyFusion_wide_PA_split_0.hdf5
Validation AUC: 0